In [2]:
import pandas as pd
import numpy as np 
import io
import re
import requests
import json
from flask import jsonify, Flask
from urllib.request import urlopen
import finnhub
import seaborn as sns
from word2number import w2n
import matplotlib.pyplot as plt
import string

In [3]:
path ="/Users/petebaker/Downloads/minfac.csv"
df = pd.read_csv(path,encoding='latin1')

In [4]:
df.head()

,position,rec_id,year,country,commodity,location,fac_name,fac_type,dmslat,dmslong,...,precision,mm,op_comp,maininvest,othinvest,status,capacity,units,notes,cite
0,1,1,2007,Australia,Alumina,"Gladstone, Queensland",Gladstone alumina refinery,Plant,24° 09' S,151° 13' E,...,F,NaN,"Queensland Alumina Ltd., operator","Rio Tinto Alcan, 80%","Rusal, 20%",Active,3850,thousand metric tons per year,NaN,847
1,2,2,2007,Australia,Alumina,"Nhulunbuy, Gove, Northern Territory",Gove alumina refinery,Plant,13° 48' S,136° 44' E,...,F,NaN,Alcan Gove Pty. Ltd.,"Rio Tinto plc, 100%",NaN,Active,3800,thousand metric tons per year,NaN,847
2,3,3,2007,Australia,Alumina,"20 kilometers northwest of Collie, Western Aus...",Worsley alumina refinery,Plant,34° 46' S,116° 04' E,...,F,NaN,"Worsley Alumina Pty. Ltd., manager","BHP Billiton Ltd., 86%",Japan Alumina Associates (Australia) Pty. Ltd....,Active,3700,thousand metric tons per year,NaN,847
3,4,4,2007,Australia,Alumina,"Pinjarra, Western Australia",Pinjarra alumina refinery,Plant,33° 22' S,115° 55' E,...,F,NaN,Alcoa World Alumina Australia,"Alcoa World Alumina Australia, 100%",NaN,Active,3400,thousand metric tons per year,NaN,847
4,5,5,2007,Australia,Alumina,"Waroona, Western Australia",Wagerup alumina refinery,Plant,33° 05' S,115° 55' E,...,F,NaN,Alcoa World Alumina Australia,"Alcoa World Alumina Australia, 60%","Western Mining Corp., 40%",Active,2600,thousand metric tons per year,NaN,847


In [5]:
x=(df[df['commodity']=='phosphate']).count()
print(x)

position      20
rec_id        20
year          20
country       20
commodity     20
location      20
fac_name       0
fac_type      20
dmslat         5
dmslong        5
latitude      20
longitude     20
precision      0
mm             6
op_comp       20
maininvest     0
othinvest      0
status        20
capacity      14
units         20
notes          0
cite          20
dtype: int64


In [6]:
df.columns

Index(['position', 'rec_id', 'year', 'country', 'commodity', 'location',
       'fac_name', 'fac_type', 'dmslat', 'dmslong', 'latitude', 'longitude',
       'precision', 'mm', 'op_comp', 'maininvest', 'othinvest', 'status',
       'capacity', 'units', 'notes', 'cite'],
      dtype='object')

In [7]:
investors = df['maininvest']

In [8]:
investors = pd.DataFrame(investors)
investors.head(100)

,maininvest
0,"Rio Tinto Alcan, 80%"
1,"Rio Tinto plc, 100%"
2,"BHP Billiton Ltd., 86%"
3,"Alcoa World Alumina Australia, 100%"
4,"Alcoa World Alumina Australia, 60%"
...,...
95,"Centennial Coal Co. Ltd., 85%"
96,"BHP Billiton Ltd., 100%"
97,"Gloucester Coal Ltd., 100%"
98,"Gujarat NRE Resources NL, 100%"


In [10]:
#removing characters from investor column
investors = df['maininvest'].str.replace(',', '')\
.str.replace('.', '')\
.str.replace('\d+', '')\
.str.replace("%","")\
.str.replace(";","")

In [31]:
investors.value_counts()

Government of India                    60
Government                             54
BHP Billiton Ltd                       23
Government of Indonesia                21
Unocal Thailand Ltd                    19
                                       ..
Highland Gold Mining Limited            1
Lionore Mining International Ltd        1
Duferco Group                           1
Cambrian Mining Plc                     1
The Griffin Coal Mining Co Pty Ltd      1
Name: maininvest, Length: 690, dtype: int64

In [20]:
# counting the duplicates
dups = df.pivot_table(index = ['maininvest'], aggfunc ='size')
  
# displaying the duplicate Series
print(dups)

maininvest
(BHP Billiton Ltd., 50%, and Mitsubishi Development Pty. Ltd., 50%)     1
(EZOTsM)                                                                1
50% owned by Orascom Construction Industries of Egypt                   1
51% owned by Luanhe Industrial Group and another unnamed Chinese Co.    1
87% controlled, directly or indirectly by Vimetco NV                    1
                                                                       ..
Zapadnaya, a/s                                                          1
Zhuye Torch Metals Co. Ltd.                                             1
Zinifex Ltd., 100%                                                      1
Zolotaya, ZDK, ZAO                                                      1
private interests, 100%                                                 1
Length: 771, dtype: int64


In [ ]:
company_symbol = {}
notListed= {}
finnhub_client = finnhub.Client(api_key="c1url52ad3ibbva6nr3g")
for investor in investors:
    try:
        company_symbol.update(finnhub_client.symbol_lookup(f'{investor}'))
        print("company listed", investor)
    except:
        notListed.update(investor)
        print("company not listed", investor)
    

company listed Rio Tinto Alcan 
company listed Rio Tinto plc 
company listed BHP Billiton Ltd 
company listed Alcoa World Alumina Australia 
company listed Alcoa World Alumina Australia 
company listed Alcoa World Alumina Australia 
company listed Rio Tinto Alcan Inc 
company listed Rio Tinto Alcan Inc ;
company listed Rio Tinto Alcan Inc 
company listed Alcoa of Australia 
company listed Alcoa of Australia 
company listed Hydro Aluminum Kurri Kurri Pty Ltd 
company listed Rio Tinto Alcan Inc 
company listed Straits Resources Ltd 
company listed Cambrian Mining Plc 
company listed Alcoa World Alumina Australia 
company listed Rio Tinto plc 
company listed BHP Billiton Ltd ;
company listed Rio Tinto plc 
company listed Rio Tinto Alcan Inc 
company listed Unimin Australia Ltd 
company listed PCP Douglas Pty Ltd 
company listed Unimin Australia Ltd 
company listed Arumpo Bentonite Pty Ltd 
company listed Cement Australia Ltd 
company listed Blue Circle Southern Cement Ltd 
company listed 

In [11]:
company_symbol

{'count': 19,
 'result': [{'description': 'NANOSONICS LTD',
   'displaySymbol': 'NAN.AX',
   'symbol': 'NAN.AX',
   'type': 'Common Stock'},
  {'description': 'NUVEEN NY QUAL MUNI INC FD',
   'displaySymbol': 'NAN',
   'symbol': 'NAN',
   'type': 'Closed-End Fund'},
  {'description': 'NORTH AMERICAN NICKEL INC',
   'displaySymbol': 'NAN.V',
   'symbol': 'NAN.V',
   'type': 'Common Stock'},
  {'description': 'NORTH AMERICAN NICKEL INC',
   'displaySymbol': 'NAN.NE',
   'symbol': 'NAN.NE',
   'type': 'Common Stock'},
  {'description': 'SPDR S&P NORTH AMERICAN NATU',
   'displaySymbol': 'NANR',
   'primary': None,
   'symbol': 'NANR',
   'type': 'ETP'},
  {'description': 'NANOPHASE TECHNOLOGIES CORP',
   'displaySymbol': 'NANX',
   'primary': [],
   'symbol': 'NANX',
   'type': 'Common Stock'},
  {'description': 'NANOVATION MICROTECH INC',
   'displaySymbol': 'NANN',
   'primary': [],
   'symbol': 'NANN',
   'type': 'Common Stock'},
  {'description': 'Nexans',
   'displaySymbol': '0IGF.L'

In [12]:
df['commodity'].unique()

array(['Alumina', 'Aluminum', 'Antimony', 'Bauxite', 'Bentonite',
       'Cement', 'Chromite', 'Coal: black', 'Coal: brown', 'Cobalt',
       'Copper', 'Diamond', 'Diatomite', 'Dolomite', 'Feldspar', 'Garnet',
       'Gas: condensate', 'Gas: natural', 'Gas: natural, liquefied',
       'Gold', 'Gypsum', 'Iron ore', 'Kaolin', 'Lead', 'Magnesite',
       'Manganese', 'Manganese alloys', 'Mineral sands', 'Molybdenum',
       'Nickel', 'Opal', 'Petroleum', 'Phosphate rock', 'Pig iron',
       'Salt', 'Silica', 'Silver', 'Spodumene', 'Steel', 'Talc',
       'Tantalum: tantalite (Ta2O5)', 'Tin', 'Tungsten', 'Uranium',
       'Vanadium', 'Zinc', 'Coal', 'Fertilizer', 'Granite',
       'Petroleum: crude', 'Petroleum: refined', 'Ferrosilicon',
       'Gemstone', 'Limestone', 'Asbestos', 'Barite', 'Bismuth',
       'Cadmium', 'Copper: refined', 'Gallium', 'Germanium',
       'Gold: refined', 'Graphite', 'Indium',
       'Iron and steel: crude steel', 'Iron and steel: ferroalloys',
       'Iron an

In [1]:
mineralOutput = []
for item in enumerate(capacity):
    x = item.split
    output.append(cap)
    
    



NameError: name 'capacity' is not defined

In [177]:
output = pd.DataFrame(mineralOutput)

In [179]:
output[1].unique()


array(['3850', '3800', '3700', ..., '79000 Pyr:', '120000 sulf', '342'],
      dtype=object)

In [166]:
df.dtypes

position        int64
rec_id          int64
year            int64
country        object
commodity      object
location       object
fac_name       object
fac_type       object
dmslat         object
dmslong        object
latitude      float64
longitude     float64
precision      object
mm             object
op_comp        object
maininvest     object
othinvest      object
status         object
capacity       object
units          object
notes          object
cite            int64
dtype: object